# Big Data Notebook

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
taxi_df = pd.read_csv('../sample_data/taxi_sample.csv')
spark = SparkSession.builder.master('local[*]').appName('TaxiBigData').getOrCreate()
spark_df = spark.createDataFrame(taxi_df)
spark_df.show(5)